In [98]:
import gradio as gr
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [99]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
assert enc.decode(enc.encode("hello world")) == "hello world"

# To get the tokeniser corresponding to a specific model in the OpenAI API:
print(enc.encode("hello world"))

[31373, 995]


In [100]:
pre_prompt = "Act like you are an AI. Answer short, just one sentence if possible."

print(pre_prompt)

messages=[
          {"role": "system", "content": pre_prompt},
      ]

history_tokens = enc.encode(pre_prompt)

Act like you are an AI. Answer short, just one sentence if possible.


In [101]:
def chat_response(messages):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", max_tokens=100, temperature=0.8, messages=messages
    )
    content = response["choices"][0]["message"]["content"]
    return content


def construct_messages_list(prompt, messages=[]):

    print("You: ", prompt)

    messages.append({"role": "user", "content": prompt})
    history_tokens.extend(enc.encode(prompt))

    response = chat_response(messages)
    print("Bot: ", response)

    messages.append({"role": "assistant", "content": response})
    history_tokens.extend(enc.encode(response))

    #print(messages)
    #print(history_tokens)

    return messages, history_tokens


In [102]:
def in_editor_chat(messages):
    prompt = ""
    while prompt != "exit":
        prompt = input("Chat: ")
        construct_messages_list(prompt, messages)

in_editor_chat(messages)

You:  hello
Bot:  Hello! How can I assist you today?
You:  code
Bot:  What programming language do you want to use for coding?
You:  uuuh
Bot:  Do you need help with something specific related to coding?
You:  python
Bot:  Python is a high-level, interpreted programming language used for web development, data analysis, artificial intelligence, and many other purposes.
You:  CATS
Bot:  Cats are feline animals known for their agility, curiosity, and companionship with humans.
You:  YYYEEE
Bot:  I'm sorry, I don't understand what you mean. Could you please provide more context or information?
You:  col
Bot:  "col" is a shorthand for "column" and is commonly used in programming and data analysis to refer to individual columns of data within a dataset or spreadsheet.
You:  exit
Bot:  Goodbye, have a nice day!


In [103]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    state = gr.State([])

    with gr.Row():
        txt = gr.Textbox(
            show_label=False, placeholder="Enter text and press enter"
        ).style(container=False)

    txt.submit(construct_messages_list, [txt, state], [chatbot, state])

demo.launch()


Running on local URL:  http://127.0.0.1:7875

To create a public link, set `share=True` in `launch()`.
